<a href="https://colab.research.google.com/github/sadra-barikbin/persian-information-retrieval-example/blob/unify-all-methods/Persian-IR-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install hazm transformers ir_measures
!pip install -q clean-text[gpl]

<div dir='rtl'>
    ابتدا کتاب‌خانه‌های لازم را فرا می‌خوانیم
</div>

In [ ]:
import torch
import yaml
import hazm
import tqdm
import numpy as np
import pandas as pd
import ir_measures as IRm
import tensorflow
import itertools
from torch.data.utils import DataLoader,IterableDataset
from typing import List, Tuple, Union
from pathlib import Path
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# Loading & Preparing Data

## Corpus

In [ ]:
!wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip

In [ ]:
!unzip doc_collection.zip

In [ ]:
!cat IR_dataset/1000.txt

<div dir='rtl'>
    در این بخش مجموعه دادگان دانلود و استخراج می‌شود و سپس از روی آن
    Corpus
    برای ساخت مدل‌های زبانی ایجاد می‌کنیم
</div>

In [ ]:
# corpus = [(int(path.stem), path.open().read()) for path in Path('dataset/IR_dataset').iterdir()]
corpus = [(int(path.stem), path.open().read()) for path in Path('dataset/IR_dataset').iterdir()]
corpus = pd.DataFrame(corpus, columns=['docId','text']).set_index('docId').sort_index()

In [ ]:
ccorpus = [(int(path.stem), path.open().read()) for path in Path('IR_dataset').iterdir()]

In [ ]:
corpus.head()

,text
docId,
0,برخی از هواداران مصدق یا اعضای جبهه ملی که در ...
1,جبهه ملی ایران که به اختصار جبهه ملی نیز خواند...
2,سرلشکر زاهدی در سال ۱۳۲۸ و پس از آن‌که دخالت‌ه...
3,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...
4,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...


## Qrels

<div dir='rtl'>
    در این بخش فایل کوئری‌ها را دانلود می‌کنیم و از روی فایل آن دیتافریمی می‌سازیم که کوئری و داک بازیابی شده با سطح نزدیکی را نمایش دهد
</div>

In [ ]:
!wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml

In [ ]:
# query_raw_data = yaml.safe_load(open('evaluation_IR.yml'))
query_raw_data = yaml.safe_load(open('dataset/evaluation_IR.yml'))

In [ ]:
query = pd.Series(query_raw_data.keys())
qrels = [{'query_id':idx, 'doc_id':d,
          'relevance':3} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_high']]
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':2} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_med']])
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':1} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_low']])
qrels.extend([{'query_id':idx, 'doc_id':query_raw_data[q]['relevant'][0],
          'relevance':4} for idx,q in query.to_dict().items()])
qrels = pd.DataFrame(qrels)

In [ ]:
query[147],query_raw_data[query[147]]

('گرجستان  تاریخ',
 {'relevant': [388],
  'similar_high': [389, 390, 391, 392, 393, 394],
  'similar_low': [404, 405, 406, 407, 408, 409, 410, 411, 412, 413],
  'similar_med': [395, 364, 396, 397, 398, 399, 400, 401, 402, 403]})

In [ ]:
qrels.sample(n=5).reset_index(drop=True)

,query_id,doc_id,relevance
0,126,464,2
1,129,1083,2
2,122,2137,3
3,48,2613,2
4,119,617,1


## Normaliztion

<div dir='rtl'>
    با استفاده از نرمال‌ساز کتابخانه هضم، فایل‌های خود را نرمال می‌کنیم که در فرآیند ساخت مدل زبانی بهتر عمل کنیم
</div>

In [ ]:
normalize = hazm.Normalizer().normalize
corpus.text = corpus.text.transform(normalize)
query = query.transform(normalize)

# Embedding the documents

## Method 1 : Tfidf

<div dir='rtl'>
    برای ساخت بردارها بر اساس
    Tf-Idf
    از کتاب‌خانه
    sklearn
    استفاده می‌کنیم
</div>

In [ ]:
vectorizer = TfidfVectorizer(max_features=500,ngram_range=(1,2))
vectorizer.fit(corpus.text)

TfidfVectorizer(max_features=500, ngram_range=(1, 2))

## Method 2 : ParsBert

<div dir='rtl'>
    برای استفاده از برت، از
    ParsBert
    استفاده می‌کنیم.
</div>

In [ ]:
class BertVectorizer(TransformerMixin):
  def __init__(self):
    self.model = AutoModel.from_pretrained("HooshvareLab/bert-fa-zwnj-base", from_tf = True).cuda()
    self.tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")
    self.model.eval()
    self.fitted = False
  def get_embed(self, X: Union[List[str],pd.Series]):
    result = torch.empty((0,768))
    for batch in DataLoader(IterableDataset(X),batch_size=128,shuffle=False):
      encoding = tokenizer.encode_plus(batch,add_special_tokens=True,return_token_type_ids=False,max_length = 500,
                                       truncation=True,return_attention_mask=True,return_tensors='pt')
      encoding = {k:v.cuda() for k,v in encoding.items()}
      with torch.no_grad():
        out = self.model(**encoding)
      result = torch.cat(result, out['pooler_output'].cpu())
    return result.numpy()
  def fit(self,X: Union[List[str],pd.Series]):
    if self.fitted:
      return
    doc_vec = np.empty((0, 768))
    doc_map = np.empty(0)
    tokenizer = hazm.WordTokenizer(replace_numbers=True)
    subdocs = []
    subdoc_doc_idx = []
    for index, doc in tqdm.tqdm(X):
      doc_split = tokenizer.tokenize(doc)
      doc_parts = [' '.join(doc_split[i:i + 300]) for i in range(0, len(doc_split) - 150, 150)]
      subdocs.extend(doc_parts)
      subdoc_doc_idx.extend([index] * len(doc_parts))
    
      for part in doc_parts:
        doc_vec = np.append(doc_vec, get_embed(part), axis = 0)
        doc_map = np.append(doc_map, index)
    self.vectors = result
    self.fitted = True
  def transform(self, X)


In [ ]:
text = "ما در قرن ۲۱ زندگی می‌کنیم" 
encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      max_length = 500,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
    )
out = model(
            input_ids = encoding['input_ids'].cuda(), 
            attention_mask= encoding['attention_mask'].cuda())
out['pooler_output'][0]

<div dir='rtl'>
    ابتدا تنظیمات لازم را برای استفاده از مدل از پیش آموزش داده شده تنظیم می‌کنیم.
    سپس از آنجا که طول داک‌ها بعضا بیشتر از ۵۰۰ کلمه هستند،
    هر فایل را به پنجره‌های ۳۰۰ کلمه‌ای تقسیم می‌کنیم،
    به طوری که هر کلمه در دو پنجره ظاهر شود و با این کار برای هر فایل ممکن است چند بردار تولید شود
    نهایتا نزدیک‌ترین بردار آن به کوئری را به عنوان پاسخ آن در نظر می‌گیریم؛
    اما گویا به طور کلی از برت برای داده‌های بزرگ استفاده نمی‌شود
</div>

In [ ]:
def get_embed(part):
  encoding = tokenizer.encode_plus(
    part,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    max_length = 500,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
  )
  encoding = {k:v.cuda() for k,v in encoding.items()}
  with torch.no_grad():
    out = model(**encoding)
  return out['pooler_output'].cpu().numpy()

In [ ]:
doc_vec = np.empty((0, 768))
doc_map = np.empty(0)
tokenizer = hazm.WordTokenizer(replace_numbers=True)
for index, doc in tqdm.tqdm(corpus.iterrows()):
  doc_split = tokenizer.tokenize(doc['text'])
  doc_parts = [' '.join(doc_split[i:i + 300]) for i in range(0, len(doc_split) - 150, 150)]
  for part in doc_parts:
    doc_vec = np.append(doc_vec, get_embed(part), axis = 0)
    doc_map = np.append(doc_map, index)

3258it [06:04,  8.95it/s]


<div dir='rtl'>
    پیاده‌سازی روش سوم که
    tf-idf
    وزن‌دار برحسب
    pos-tag
    هاست در نوتبوکی رو فولدر
    method-3
    آمده‌است
</div>

# Document Retrieval

<div dir='rtl'>
    با استفاده از تابع نزدیک‌ترین همسایه‌ها در
    sklearn
    کلاسی برای اجرای الگوریتم
    KNN
    می‌سازیم که داک‌ها را
    fit
    کند و داک‌های نزدیک کوئری را حدس بزند.
    برای هر دو روش برت و
    tf-idf
    از
    knn
    استفاده می‌کنیم
</div>

In [ ]:
class KNN_based_IR(BaseEstimator):
  def __init__(self,sampleIdx_docId_map=None, n_neighbors=1+10+10+10) -> None:
    self.sampleIdx_docId_map = sampleIdx_docId_map
    self.n_neighbors = n_neighbors
    self.nn = NearestNeighbors(n_neighbors=n_neighbors)
  def set_params(self,**kwargs):
    self.nn.set_params(**kwargs)
  def fit(self, X: np.array):
    self.nn.fit(X)
  def predict(self, X: np.array):
    distances, sampleIndices = self.nn.kneighbors(X, n_neighbors=self.n_neighbors)
    scores = np.max(distances)-distances
    if not self.sampleIdx_docId_map:
      docIds = sampleIndices
    else:
      docIds = np.array([self.sampleIdx_docId_map[idx] for idx in sampleIndices.flatten()]).reshape(sampleIndices.shape)
      _scores = []
      _docIds = []
      for i in range(docIds.shape[0]):
        docs_of_query = docIds[i]
        _docIds.append(list(dict(zip(docs_of_query, np.empty(len(docs_of_query)))).keys()))
        doc_score_dict = {}
        for j in range(len(docs_of_query)):
          docId = docs_of_query[j]
          if docId in doc_score_dict:
            continue
          else:
            doc_score_dict[docId] = scores[i][j]
        _scores.append(list([score for doc,score in doc_score_dict.items()))
      docIds = _docIds
      scores = _scores
    return scores, docIds

In [ ]:
IR_system = KNN_based_IR()
IR_system.fit(vectorizer.transform(corpus.text))

In [ ]:
bert_knn = KNN_based_IR(n_neighbors=80)
bert_knn.fit(doc_vec)

# IR Evaluation

In [ ]:
def tf_knn_pred(knn):
  return knn.predict(vectorizer.transform(query))


preds = tf_knn_pred(IR_system)

<div dir='rtl'>
    برای تخمین برت برحسب
    knn
    باید تعداد بیشتری بردار نزدیک را به دست بیاوریم و بردارهای مربوط به یک داک یکسان را از روی آن حذف کنیم
    و فقط نزدیک‌ترین را باقی بگذاریم
</div>

In [ ]:
def bert_knn_pred(knn):
  bert_score = []
  bert_id = []
  mn = 100
  for q in tqdm.tqdm(query):
    score, oid = knn.predict(get_embed(q))
    score = score[0]
    doc_id = [doc_map[i] for i in oid[0]]
    n_score = []
    n_id = []
    for sc, id in zip(score, doc_id):
      if id not in n_id:
        n_id.append(id)
        n_score.append(sc)
    mn = min(mn, len(n_score[:31]))
    bert_score.append(n_score[:31])
    bert_id.append(n_id[:31])
  
  return (np.array(bert_score), np.array(bert_id).astype(int))


bert_pred = bert_knn_pred(bert_knn)

100%|██████████| 150/150 [00:06<00:00, 24.19it/s]


## Adapting IR output to our Test Collection

<div dir='rtl'>
    از تابع زیر برای اینکه بتوانیم خروجی بازیابی را تبدیل به حالتی برای ارزیابی کوئری‌ها بکنیم استفاده می‌کنیم.
    ما برای اینکه بتوانیم خروجی
    knn
    را تقریبا نزدیک به خروجی کوئری‌ها کنیم، تقریب زدیم که غیر از داک اصلی،
    از هر سطح نزدیکی به طور میانگین ۱۰ داک بازیابی می‌شود و با استفاده از آن ملاک‌ها را اندازه می گیریم
</div>

In [ ]:
def adapt_IR_output_to_measure_input(IR_output: Tuple[Union[np.array,List[List[float]]], Union[np.array,List[List[int]]]]):
  scores, docIds = IR_output
  if type(docIds) == list:
    return pd.DataFrame({'query_id': list(itertools.chain(*[[query.index[i]]*len(docIds[i]) for i in range(len(query.index))])).astype(str),
                       'doc_id':   list(itertools.chain(*docIds)).astype(str),
                       'score':    list(itertools.chain(*scores))})
  else:
    return pd.DataFrame({'query_id': np.tile(query.index,(31,1)).flatten(order='F').astype(str),
                       'doc_id':   docIds.flatten().astype(str),
                       'score':    scores.flatten()})

<div dir='rtl'>
    این تابع نیز برای بهبود عملکرد
    knn
    ایجاد شده است تا با تغییر متریک آن بتوانیم بازیابی بهتری با توجه به ملاک‌های ارزیابی داشته باشیم
</div>

In [ ]:
def knn_tuning(k, param, embed, pred_f, measure):
  score = -1
  best_p = -1
  for p in param:
    knn = KNN_based_IR(n_neighbors=k)
    knn.set_params(metric = p)
    knn.fit(embed)
    val = measure(qrels.astype({'query_id':str,'doc_id':str}),pred_f(knn))
    if val > score:
      score = val
      best_p = p
  return best_p, score


param = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan'] # Metrics for sparse input

## MRR (Mean Reciprocal Rank)

<div dir='rtl'>
    برای استفاده از این اندازه‌گیری، باید محل مرتبط‌ترین داک هر کوئری را بین لیست داک‌های بازیابی شده بیابیم و معکوس آن را حساب کنیم و سپس میانگین بگیریم.
</div>

In [ ]:
MRR = IRm.measures.MRR()
def mrr_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return MRR.calc_aggregate(qrels[qrels.relevance == 4], ret)
# mrr_scorer = make_scorer(mrr)

In [ ]:
mrr_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.11047063325391496

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, mrr_measure)

('cosine', 0.11664736491005835)

In [ ]:
mrr_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.06931617823372208

In [ ]:
knn_tuning(80, param, doc_vec[1:], bert_knn_pred, mrr_measure)

100%|██████████| 150/150 [00:05<00:00, 25.57it/s]


('cosine', 0.06982707206877474)

## MAP (Mean Average Precision)

In [ ]:
def map_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.AP(rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret) for level in range(1,4+1)])

# map_scorer = make_scorer(map)

In [ ]:
map_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.06630537139767945

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, map_measure)

('cosine', 0.06931719832816999)

In [ ]:
map_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.048315030270106925

In [ ]:
knn_tuning(80, param, doc_vec[1:], bert_knn_pred, map_measure)

100%|██████████| 150/150 [00:05<00:00, 25.91it/s]


('cityblock', 0.04838834768432319)

## P@K

<div dir='rtl'>
    با توجه به اینکه در کوئری‌ها ما رنکینگ نداریم، برای محاسبه این معیار
    precision
    را در
    i
    دسته اول حساب می‌کنیم و سپس میانگین آن‌ها را برمی‌گردانیم
</div>

In [ ]:
def p_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.P(cutoff=k, rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret)\
                  for k,level in zip([1,11,21,31],range(1,4+1))])

In [ ]:
p_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.033872596937113045

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, p_measure)

('cosine', 0.03418284224735836)

In [ ]:
p_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.026597076758367057

In [ ]:
knn_tuning(80, param, doc_vec[1:], bert_knn_pred, p_measure)

100%|██████████| 150/150 [00:05<00:00, 26.06it/s]


('cosine', 0.02688893543732251)

<div dir='rtl'>
    بنظر می‌رسد
    tuning
    های انجام شده تغییر ناچیزی در امتیازهای ما داشته‌اند و نتیجه را بهبود چندانی نداده‌اند
</div>

# Model Selection

In [ ]:
param_grid = {
    'embedding': [bert_vectorizer,tfidf_vectorizer],
    ''
}

In [ ]:
pipeline = Pipeline([('embedding','passthrough'),
                     ('retrieval','passthrough')])